In [1]:
#Directions

#make sure you have installed cv2 with command: 
#  pip install opencv-python
#download the file called pose_iter_584000.caffemodel which is saved in this dropbox:
#  https://www.dropbox.com/s/3x0xambj2rkyrap/pose_iter_584000.caffemodel?dl=0
#download the file called pose_deploy.prototxt which is saved here:
#  https://github.com/CMU-Perceptual-Computing-Lab/openpose/tree/master/models/pose/body_25

#save both of these files in the same folder as this ipynb
#save a video in the same folder or subfolder


#issues:

# the model keeps recognizing exercise equipment in the background as body parts,
# i think using a different model(for multiple people simultaneously), 
# or adjusting confidence could help with this

# i could only make the scaling work by cropping the images to squares first
# not a big deal now with our training data, but it could cause problems in the future

# i think most of these .avi videos are 30 fps. if not, framestep() will need to be updated

# this script does not save the video, it only displays it in a popup window. 


In [13]:
import cv2

import csv

# Open the video file
video_path = 'DeepSquat1 (1).avi'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Cannot open camera")

# Define the keypoint mapping for this OpenPose body_25 model
keypoints_mapping = {
    0:  "Nose", 1:  "Neck", 2:  "RShoulder", 3:  "RElbow", 4:  "RWrist", 5:  "LShoulder", 6:  "LElbow",
    7:  "LWrist", 8:  "MidHip", 9:  "RHip", 10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee",
    14: "LAnkle", 15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe", 20: "LSmallToe",
    21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel"
}
#load the model
net = cv2.dnn.readNetFromCaffe('pose_deploy.prototxt', 'pose_iter_584000.caffemodel')

#Write frame information of joint locations into a csv file
f = open("DeepSquatJoint.csv", 'a')
w = csv.writer(f)
w.writerow(keypoints_mapping.values())
 
def framestep(cap):
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    print("fps: %d" %(fps))
    print()
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    new_vid = []
    if (fps % 10 == 0): #we can easily do 10 fps
        stepsize = fps/0.1 # reset to 10
        print("total frames to be analyzed: ", frame_count / stepsize)
        curr_frame = 0
        i = 0
        while curr_frame < frame_count:
            ret, frame = cap.read()
            if curr_frame % stepsize == 0:
                print("starting frame: ", i)
                new_vid.append(pose(frame))
                i+=1
            curr_frame+=1
    
    else:
        print("NOT BUILT YET")
        pass
    
    return new_vid
    
def squarify(frame):
    height, width, _ = frame.shape
    min_dim = min(height, width)

    # Calculate the cropping dimensions
    crop_height = (height - min_dim) // 2
    crop_width = (width - min_dim) // 2

    # Crop the image equally from both sides to make it a square
    return frame[crop_height:crop_height+min_dim, crop_width:crop_width+min_dim] , min_dim

def pose(frame):
    
    frame, size = squarify(frame)
    blob = cv2.dnn.blobFromImage(frame, 1/255, (size, size),
                             (0, 0, 0), swapRB=False, crop=True)

    # run forward pass to get the pose estimation
    net.setInput(blob)
    output = net.forward()
    
    # Extract joint locations
    joint_locations = []

    for i in range(len(keypoints_mapping)): #-1 bc we dont want point for the background
        keypoint = output[0, i, :, :]
        min_val, confidence, min_loc, point = cv2.minMaxLoc(keypoint)

        if confidence > 0.1:  # can adjust the confidence threshold if needed ???
            joint_locations.append((8 * int(point[0]), 8 * int(point[1]), i))
        else:
            joint_locations.append(None)

    #joint_locations contains the locations of the detected joints and corresponding index

    for location in joint_locations:
        if location:
            x, y, index = location
            cv2.circle(frame, (x, y), 5, (0, 0, 255), -1)
            #cv2.putText(image, str(index), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
    #print joint locations into a csv file
    w.writerow(joint_locations)
    
    return frame


#write the information + circles onto each frame and pop up a window for each frame
my_video = framestep(cap)
size = my_video[0].shape[1], my_video[0].shape[0]
print(size)
out = cv2.VideoWriter("real.avi", cv2.VideoWriter_fourcc(*'DIVX'), 30, size)

'''for i in range(len(my_video)):
    out.write(my_video[i])
    cv2.imshow("video", my_video[i])
    cv2.waitKey(0)

cv2.destroyAllWindows()
'''
out.release()
cap.release()
f.close()

exit(1)

fps: 30

total frames to be analyzed:  1.7233333333333334
starting frame:  0
starting frame:  1
(1080, 1080)


In [12]:
#to write the frames into a video format for viewer presentation

cap = cv2.VideoCapture(video_path)

def test():
    size = (1080, 1080)
    out = cv2.VideoWriter("deep.avi", cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
    for i in range(0, 200):
        f, s = squarify(cap.read()[1])
        s
        out.write(f)
    
    out.release()

test()
cap.release()

In [7]:
arr = [('hello', 7, 'on'), (3, 5, 7)]


print(str(arr))

[('hello', 7, 'on'), (3, 5, 7)]
